---
title: "Feature Engineering in Python"
subtitle: "Applied Machine Learning"
author: "Calvin"
  
jupyter: python3

execute:
    echo: true
    cache: true
    freeze: true  # never re-render during project render
---


## Agenda

1.  Feature Engineering
2.  Variable Selection

## Import

-   Python base data stack

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

-   R compatibility

In [ ]:
import pyreadr

## ML Library

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

## Setup

-   `pyreadr` to read in an R dataset.


In [ ]:
url = 'https://cd-public.github.io/D505/dat/'
rds = 'wine.rds'
pyreadr.download_file(url + rds, rds) 
wine = pyreadr.read_r(rds)[None]      
wine.dropna(subset=['points','price'])

# Feature Engineering

## Categorical vs. Continuous

-   What is a categorical variable?
-   What is a continuous variable?
-   Why visualize at the data before modeling it?

## Categorical Example 1


In [ ]:
wine['roger'] = wine['taster_name'] == "Roger Voss"
wine['pinot_gris'] = wine['variety'] == "Pinot Gris"
means = wine.groupby(['roger','pinot_gris'])['points'].mean().reset_index()
sns.lineplot(means,x='pinot_gris',y='points',hue='roger')

## Categorical Example 2


In [ ]:
filtered = wine[wine['province']=='Oregon']
sns.lineplot(filtered.groupby('year')['price'].mean())

## Exercise

1.  Group by winery and year, Find:
    -   The average score, and 
    -   Number of reviews.
2.  Find year-on-year change in score by winery.

## Year-on-Year Change Example


In [ ]:
wino = wine.groupby(['winery', 'year']).agg(
    avg_score=('points', 'mean'),
    num_reviews=('id', 'nunique')
)
wino = wino.sort_values(by=['winery', 'year'])
wino['score_change'] = wino.groupby('winery')['avg_score'].diff()
wino = wino.dropna(subset=['score_change'])
wino.head()

## Dummy Variables

-   **What are Dummy Variables?**: 
    -   Represent categories as 0s and 1s for models.
-   **Why Use Dummy Variables?**: 
    -   Handle categorical data in numerical algorithms.
-   **Avoid Dummy Trap**: 
    -   Drop one column to prevent multicollinearity.

## Many vs Few Dummies

-   **Few Dummies**: 
    -   Simplifies models, risks losing fine-grained patterns.
-   **Many Dummies**: 
    -   Captures detailed trends, increases model complexity.
-   **Key Decision**: 
    -   Balance interpretability and predictive power.

## "fastDummies" Package

-   **Purpose**: 
    -   Quickly create dummy variables in R datasets.
-   **Key Functions**: 
    -   `dummy_cols()` adds dummy columns efficiently.
-   **Features**: 
    -   Handles multiple columns and missing data flexibly.

## Few Dummies


In [ ]:
wino = wine[['taster_name']]
wino = pd.get_dummies(wino, columns=['taster_name'])
wino = wino.iloc[:, :4]
wino.head()

## Many Dummies

- Can use `siuba` for `fct_lump` or write custom:


In [ ]:
wino = wine[['variety']]
wino = pd.get_dummies(wino, columns=['variety'])
wino.head()

## Other types of engineered categorical features...

-   Words or phrases in text
-   A given time period
-   An arbitrary numerical cut-off
-   Demographic variables

## What about numerical features?


In [ ]:
sns.histplot(wine['price'], bins=30, kde=False)

## Take the natural log


In [ ]:
wine['log_price'] = np.log(wine['price'])
sns.histplot(wine['log_price'], bins=30, kde=False)

## Standardizing

-   Create a common scale across variables.
    -   Mean-centering $x-\bar{x}$
    -   Scaling: $x/std(x)$

-   Helps reduce bias when interactions are included. 
    -   (i.e. eliminates variance inflation).

## Other transformations.

-   I use logs \> 95% of the time, standarizing \~40%.
-   There are [many other transformations](http://www.feat.engineering/numeric-one-to-many.html):
    -   YoY, QoQ, etc. (absolute and percent)
    -   log
    -   polynomial transforms
    -   lags!

## Standardize

- R was: `list(normalized = ~(scale(.) %>% as.vector))`
- Py is either the calculation or include `scipy`


```{r Normalize}
wine['std_pts'] = (wine['points'] - wine['points'].mean()) / wine['points'].std()

print(wine[['points', 'std_pts']].head())
```


- There's also built-ins.

```{.python}
# or use scipy
from scipy import stats 
wine['points'] = stats.zscore(wine['points']) 
```

## Philosophy


```{dot Philosophy}
//| echo: false
digraph G {
    
    bgcolor="#101010";

    node [
        fontcolor = "#e0e0e0",
        color = "#e0e0e0",
    ]

    edge [
        color = "#e0e0e0",
        fontcolor = "#e0e0e0"
    ]
    node [shape=circle];
    A [label="All Data"];

    node [shape=pentagon];
    B [label="Training"];
    C [label="Testing"];

    node [shape=rectangle];
    D [label="Resample 1"];
    E [label="Resample 2"];
    F [label="Resample B"];

    node [shape=ellipse];
    G [label="Analysis"];
    H [label="Assessment"];
    I [label="Analysis"];
    J [label="Assessment"];
    K [label="Analysis"];
    L [label="Assessment"];

    A -> B;
    A -> C;
    B -> D;
    B -> E;
    B -> F;
    D -> G;
    D -> H;
    E -> I;
    E -> J;
    F -> K;
    F -> L;
}
```


## Types of resampling

-   [V-fold Cross-Validation](http://www.feat.engineering/resampling.html#cv)
    -   Divides data into $k$ folds, trains on $k−1$ folds, validates on the remaining fold, for all folds.
-   [Monte Carlo Cross-Validation](http://www.feat.engineering/resampling.html#monte-carlo-cross-validation)
    -   Randomly splits data into training and validation sets multiple times, averaging results for evaluation.
-   [The Bootstrap](http://www.feat.engineering/resampling.html#the-bootstrap)
    -   Uses resampling with replacement to estimate model accuracy and variability.

## Setup the Dataframe

-   Follow [this link](https://topepo.github.io/caret) for the full documentation on caret.


In [ ]:
wino = wine.copy()
wino['fr'] = (wino['country'] == "France")
wino['cab'] = wino['variety'].str.contains("Cabernet")
wino['lprice'] = np.log(wino['price'])
wino = wino.dropna(subset=['fr', 'cab'])
wino = wino[['lprice', 'points', 'fr', 'cab']]

-   Off hand, I would've standarized points as well, but
-   We're following Jameson's code...
    -   ...who *understands the data better*.

## Split Samples

- Single line train/test split with sklearn.


In [ ]:
wino_tr, wino_te = train_test_split(wino)

## Train the model

-   Configure `train` to cross validate


In [ ]:
model = LinearRegression()
cv = RepeatedKFold(n_splits=5, n_repeats=3)
scores = cross_val_score(model,
                         wino_tr.drop(columns=['lprice']), 
                         wino_tr['lprice'], 
                         cv=cv, 
                         scoring='neg_mean_squared_error')
model.fit(wino_tr.drop(columns=['lprice']),wino_tr['lprice'])

## Train vs. test

::::: columns

::: {.column width="50%"}


In [ ]:
y_pred = model.predict(wino_tr.drop(columns=['lprice']))
mean_squared_error(wino_tr['lprice'], y_pred) ** .5

:::

::: {.column width="50%"}


In [ ]:
y_pred = model.predict(wino_te.drop(columns=['lprice']))
mean_squared_error(wino_te['lprice'], y_pred) ** .5

:::

:::::

# Feature selection

## Stepwise selection

-   **What is Stepwise Selection?**: Iterative method to add or remove predictors in a model based on statistical criteria.
-   **Types**: Forward selection starts with no predictors; backward elimination starts with all predictors; stepwise combines both.
-   **Goal**: Identify a model with strong predictive power and minimal overfitting.

## Stepwise selection is bad

Harrell (2015) provides a comprehensive indictment of the method that can be encapsulated by the statement:

> **“… if this procedure had just been proposed as a statistical method, it would most likely be rejected because it violates every principle of statistical estimation and hypothesis testing.”**

  Reference: [Harrell, F. 2015. Regression Modeling Strategies. Springer.](https://link.springer.com/book/10.1007/978-3-319-19425-7https://link.springer.com/book/10.1007/978-3-319-19425-7)

## Engineer 9 features


In [ ]:
wino = wine.copy()
top_countries = wino['country'].value_counts().nlargest(4).index
top_varieties = wino['variety'].value_counts().nlargest(4).index
wino['country'] = np.where(wino['country'].isin(top_countries), wino['country'], 'Other')
wino['variety'] = np.where(wino['variety'].isin(top_varieties), wino['variety'], 'Other')
wino['lprice'] = np.log(wino['price'])
wino = wino[['lprice', 'points', 'country', 'variety']].dropna()

wino = pd.get_dummies(wino, columns=['country', 'variety'], drop_first=True)

## Add Dummy Columns
- Careful - a destructive update to `wino`!

In [ ]:
wino = wino.loc[:, ~wino.columns.str.contains('Other')]

## Factor

In [ ]:
def do_model(df, y_name):
    df_tr, df_te = train_test_split(wino)
    model = LinearRegression()
    scores = cross_val_score(model,
                            df_tr.drop(columns=[y_name]), 
                            df_tr[y_name], 
                            cv=RepeatedKFold(n_splits=5, n_repeats=3),
                            scoring='neg_mean_squared_error')
    model.fit(df_tr.drop(columns=[y_name]),df_tr[y_name])
    y_pred = model.predict(df_tr.drop(columns=['lprice']))
    tr = mean_squared_error(wino_tr['lprice'], model.predict(df_tr.drop(columns=['lprice']))) ** .5
    te = mean_squared_error(wino_te['lprice'], model.predict(df_te.drop(columns=['lprice']))) ** .5
    return model, tr, te

## Basic Model


In [ ]:
model, tr, te = do_model(wino, 'lprice')
tr, te

## Variable Importance
-   Importance depends on model used...


In [ ]:
plt.bar(wino.columns.drop('lprice'), model.coef_)
_ = plt.xticks(rotation=90)

## Variable Importance

-   Each (linear model) coefficient has a standard error, 
    -   Measures certainty of coefficient given data.
-   For the t-statistic, 
    -   Confidence that the coefficient is different from 0
    -   We divide the coefficient by the standard error.
-   If "small" error relative to coefficient
    -   Then "big" t-statistic & high feature importance!
-   What about coefficient as variable importance?

## [Recursive Feature Elimination](https://topepo.github.io/caret/recursive-feature-elimination.html) {.smaller}

1. Tune/train the model on the training set using all predictors.
2. Calculate model performance.
3. Calculate variable importance or rankings.
4. **for** each subset size $S_i$, i = 1...S **do**
    1. Keep the $S_i$ most important variables.
    2. [Optional] Pre-process the data.
    3. Tune/train the model on the training set using $S_i$ predictors.
    4. Calculate model performance.
    5. [Optional] Recalculate the rankings for each predictor.
5. **end**
6. Calculate the performance profile over the $S_i$.
7. Determine the appropriate number of predictors.
8. Use the model corresponding to the optimal $S_i$.

## Size Drop
-   It did not seem like 2024 `r` could handle 90k wine samples.
-   Python was faster on the 90k then R on 1k.

```{r 1k}
wino <- wino[sample(nrow(wino), 1000), ]
```


## Partition Again
-   Partition

In [ ]:
wino_tr, wino_te = train_test_split(wino)

## Caret RFE


In [ ]:
model = RandomForestRegressor()
rfe = RFE(estimator=model, n_features_to_select=3)
rfe.fit(wino_tr.drop(columns=['lprice']), wino_tr['lprice'])
wino_tr.drop(columns=['lprice']).columns, rfe.ranking_

## Practical Workflow


```{dot Practice}
//| echo: false
digraph feature_engineering_pipeline {
    
    bgcolor="#101010";

    node [
        fontcolor = "#e0e0e0",
        color = "#e0e0e0",
    ]

    edge [
        color = "#e0e0e0",
        fontcolor = "#e0e0e0"
    ]
    node [shape=box];
    "Raw Data" -> "Lots of Features" [label="Feature Engineering"];
    "Lots of Features" -> "Candidate Features" [label="Feature Selection"];
    "Candidate Features" -> "Shortlist Features" [label="Expert Input"];
    "Shortlist Features" -> "Finalist Models" [label="DS Judgement"];
    "Finalist Models" -> "Production" [label="Business Unit"];
}
```


## Key Terms

::::: columns

::: {.column width="50%"}
-   Feature Engineering
-   Categorical Feature
-   Continuous Feature
-   Dummy
-   Interaction
:::

::: {.column width="50%"}
-   Sklearn
-   Model
-   Resampling
-   Train vs. Test Data
-   Variable Importance
:::

:::::